In [986]:
# Settings
_path_to_settings_files = "C:\\Users\\Stephen\\OneDrive\\Documents\\NCI\\Thesis\\Data"

# Path to source files
_path_to_files = "C:\\BBC_Data\\bbc-fulltext\\bbcAll"
#_path_to_files = "C:\\BBC_Data\\bbc-sportext\\bbcAll"

# Path to outputs
_path_to_output = "C:\\BBC_Data\\bbc-fulltext"
#_path_to_output = "C:\\BBC_Data\\bbc-sportext"

# in_degree or degree
_in_degree = True

# Window size to use for graph-of-words extraction
_windowSize = 3

# If the degree is > windowSize-1 then add the term and edge
# We use windowSize for in_degree counts, and windowSize*2 for degree counts
_degree_compare = (_windowSize-1)*2
if _in_degree == True:
    _degree_compare = _windowSize-1

# Limits for connected term removal
_upper_percentile = 99.7
_lower_count = 2


_Test_Run = 0

In [292]:
# Import required libraries

# Regex
import re

#import nltk
from nltk.corpus import stopwords

from nltk.stem.porter import *
stemmer = PorterStemmer()

# NetworkX graph library
import networkx as nx

# Random library
import random

# File libraries
import os
import glob
from pathlib import Path

# Time Library
from time import time

# Maths library
import math

# Scipy stats functions
import scipy.stats
import numpy as np

# KD Tree algo
from sklearn.neighbors import KDTree

# kmeans cluster algorithm 
from sklearn.cluster import KMeans

# Confusion matrix, precision, recall, F1
from sklearn.metrics import *

# Pandas library
import pandas as pd

# Operator
import operator

In [293]:
# Get list of Google stopwords from file

fname = _path_to_settings_files + "\\GoogleStopwords.txt"

with open(fname) as f:
    StopWords = f.readlines()
# remove whitespace characters like `\n` at the end of each line
StopWords = [x.strip() for x in StopWords]
# Stem the stopwords
StopWords = list(set([stemmer.stem(y) for y in StopWords]))



In [294]:
# Create list of stop words to use- first the NLTK stopwords
cachedStopWords = stopwords.words("english")
# Add in the Google Stopwords
cachedStopWords += StopWords
#print(cachedStopWords)

In [295]:
# VB style text parsing functions
def left(s, amount):
    return s[:amount]

def right(s, amount):
    return s[-amount:]

def mid(s, offset, amount):
    return s[offset:offset+amount]


In [296]:
# This function cleans a piece of text of non letter/space characters
p1 = re.compile(r'[^a-z ]', re.UNICODE)
p2 = re.compile(r' +', re.UNICODE)

def CleanWord(w):
    x = w.lower()
    #x = re.sub(r'[^a-z ]','',x)
    x = p1.sub(' ', x) #.strip() 
    x = p2.sub(' ', x).strip()
    #x = x.split(' ')
    return x

print(CleanWord("the quick Quick \r\nbrown.\r\nfox 1.2)"))

the quick quick brown fox


In [297]:
import csv

def StoreGraphToFile(Graph, PassNum, Stage, Desc):
    passNo = right('00' + str(PassNum), 2)
    filename = _path_to_output + '\\Pass' + passNo + '.' + str(Stage) + ' ' + Desc + '.csv'
    myfile = open(filename, 'w')
    wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

    for n,d in Graph.nodes_iter(data=True):
        mylist=str(n) + "\t" + str(d["type"]) + "\t" + str(d["x"]) + "\t" + str(d["y"]) + "\n"
        myfile.write(mylist)

    myfile.close()

In [298]:


# Function to parse out words and bigrams
def FileToGraph(fileTuple):
    # Array to return from function
    rArr = []

    # Take the filename from the tuple
    filename = fileTuple[0]
    
    # remove non letters from the text in the tuple
    r = CleanWord(fileTuple[1])
    
    # split into an array, stem, and remove words in the stopword list
    arr = [stemmer.stem(y) for y in r.split(" ")]
    arr = [word for word in arr if word not in cachedStopWords]
    
    # Create a new directed graph
    G = nx.DiGraph()
    
    arrLen = len(arr)

    # Add the first few nodes
    for i in range(0, _windowSize-1):
        G.add_node(arr[i])

    for i in range(_windowSize-1, arrLen):
        G.add_node(arr[i])
        for j in range(0, (_windowSize-1)):
            src = arr[i-(_windowSize-1)]
            tgt = arr[i-j]
            if G.has_edge(src, tgt):
                # we added this one before, just increase the weight by one
                G[src][tgt]['weight'] += 1
            else:
                # new edge. add with weight=1
                G.add_edge(src, tgt, weight=1)

    for i in range(arrLen-(_windowSize-1), arrLen):
        for j in range(1, (arrLen-i)):
            src = arr[i]
            tgt = arr[i+j]
            if G.has_edge(src, tgt):
                # we added this one before, just increase the weight by one
                G[src][tgt]['weight'] += 1
            else:
                # new edge. add with weight=1
                G.add_edge(src, tgt, weight=1)
        
    # Get the "indegree" or "degree" of the terms
    if _in_degree == True:
        d = G.in_degree()
    else:
        d = G.degree()
    

    # Now add all the words the the return array with the filename and the count
    for term in d:
        rArr.append(filename + "\t" + term + "\t" + str(d[term]))
        
    # Return the array of graphs and the array of words
    return rArr
    #return G



In [299]:

# LOCAL PYTHON IMPLEMENTATION:
def GraphTextFiles(p):

    # If this is a folder, we need to append *.* for glob
    if(os.path.isdir(p)==True):
        p+="\\*.*"
    
    f_list = glob.glob(p)
    rVal = []
    testSet = []
    
    # Get all the files - one file at a time
    for f in f_list:
        contents = Path(f).read_text()
        # Hive off a random 10% for a test set
        if random.random() < .1:
            testSet += FileToGraph((f, contents))
        else:
            rVal += FileToGraph((f, contents))
        
    return rVal, testSet



In [987]:
##### MAIN PROCESS STARTS HERE #####
_Test_Run += 1

t0 = time()

text_files, test_set = GraphTextFiles(_path_to_files + "\\????.txt")

# PYSPARK IMPLEMENTATION: 
#text_files = sc.wholeTextFiles("C:\\BBC_Data\\bbc-fulltext\\bbcAll\\*.txt") \
#    .flatMap(lambda fileTuple: FileToGraph(fileTuple))
#text_files.saveAsTextFile("C:\\BBC_Data\\Output\\files")

print("done in %0.3fs." % (time() - t0))


done in 84.844s.


In [988]:
print(len(test_set)/len(text_files) * 100)

10.637140806160406


In [989]:
############################################################################################################
############################################################################################################

_KL_Limit = 0.0000000250

############################################################################################################
############################################################################################################

# Create the initial graph
Gr=nx.Graph()
GTest=nx.Graph()

# fArr will hold the array of filenames
fArr = []
fTest = []

# tArr will hold the array of terms added to the graph
tArr = []
tTest = []

# tArr1 will hold the array of terms that we have seen at least once
tArr1 = []

t0 = time()

for x in text_files:   # FOR SPARK APPEND: .collect():

    # Split out the 
    y = x.split("\t")
    filename = y[0]
    term = y[1]
    degree = y[2]
    label = filename[32:33]
    
    # If the filename is not already added, add it to the graph
    if not filename in fArr:

        # Create some random spatial positions
        xR = random.random()*1000
        yR = random.random()*1000
        
        # Add the file node to the graph
        Gr.add_node(filename, type="file", x=xR, y = yR, label = label)
        
        # Add the filename to the file array
        fArr.append(filename)
        
    # if the degree is over the limit
    if int(degree)>_degree_compare:
        # If we have not seen this term before, add a node to the graph
        if not term in tArr:
            # Create some random spatial positions
            xR2 = random.random()*1000
            yR2 = random.random()*1000

            # Add the term to the graph
            Gr.add_node(term, type="term", x=xR2, y=yR2)
            tArr.append(term)
            
        # Add the edge to the graph
        Gr.add_edge(filename, term, weight=degree)

for x in test_set:

    # Split out the 
    y = x.split("\t")
    filename = y[0]
    term = y[1]
    degree = y[2]
    label = filename[32:33]
    
    # If the filename is not already added, add it to the test Graph
    if not filename in fTest:

        # Add the file node to the graph
        GTest.add_node(filename, type="file", label = label)
        
        # Add the filename to the file array
        fTest.append(filename)
        
    # if the degree is over the limit
    if int(degree)>_degree_compare:
        # If we have not seen this term before, add a node to the graph
        if not term in tTest:
            # Add the term to the graph
            GTest.add_node(term, type="term")
            tTest.append(term)
            
        # Add the edge to the graph
        GTest.add_edge(filename, term, weight=degree)

print("done in %0.3fs." % (time() - t0))


done in 24.286s.


In [990]:
# Remove terms with only one connection or > % fractile # of connections
t0 = time()
#d = Gr.in_degree()
d = Gr.degree()

## Store degrees to file
#filename = _path_to_output + '\\term_degree.csv'
#myfile = open(filename, 'w')
#wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

# List to allow calculation of percentile
degreeList = []

for t in tArr:
    degreeList.append(d[t])
    #myfile.write(t + "\t" + str(d[t]) + "\n")


#myfile.close()

# What #degree is > 99.7    
perc = np.percentile(degreeList, _upper_percentile)

for t in tArr:
    n = d[t]
    # Remove <2 #####or > 99.7 pecentile
    if n < _lower_count or n > perc:
        # Remove Edge
        for f in Gr.neighbors(t):
            Gr.remove_edge(f,t)
        # Remove node
        Gr.remove_node(t)
        # Remove from tArr list
        tArr.remove(t)
        
print("done in %0.3fs." % (time() - t0))


done in 0.479s.


In [991]:
# Dictionaries for calculating Kullback-Leibler
Px = []
Qx = []
Py = []
Qy = []

# Load the Q arrays with the x/y probabilities
t0 = time()
for f in fArr:
    Qx.append(Gr.node[f]["x"])
    Qy.append(Gr.node[f]["y"])

print("done in %0.3fs." % (time() - t0))


done in 0.003s.


In [992]:

# Initial values for Kullback-Leibler
KLx = 999
KLy = 999

t0 = time()

# Run several itterations of moving points
#for i in range(0, 20):
i = 0
#while KLx + KLy > 0.0000000125:
while KLx + KLy > _KL_Limit:
    
    # Reposition all the term nodes in the centroid of their documents
    for t in tArr:
        sumx=0
        sumy=0
        sumw=0

        for f in Gr.neighbors(t):
            w=float(Gr.edge[f][t]["weight"])
            sumw+=w
            sumx+=float(Gr.node[f]["x"])*w
            sumy+=float(Gr.node[f]["y"])*w

        if sumw > 0:
            Gr.node[t]["x"] = sumx/sumw
            Gr.node[t]["y"] = sumy/sumw
        else:
            print("0 weight on term " + t)

    StoreGraphToFile(Gr, i, 1, "Reposition Terms")
    
    pSumx = 0
    pSumy = 0
    
    # Reposition all the file nodes in the centroid of their terms
    for f in fArr:
        sumx=0
        sumy=0
        sumw=0

        for t in Gr.neighbors(f):
            w=float(Gr.edge[f][t]["weight"])
            sumw+=w
            sumx+=float(Gr.node[t]["x"])*w
            sumy+=float(Gr.node[t]["y"])*w

        if sumw > 0:
            Gr.node[f]["x"] = sumx/sumw
            pSumx += sumx/sumw
            Gr.node[f]["y"] = sumy/sumw
            pSumy += sumy/sumw
        else:
            print("0 weight on file " + f)

    # Load the P arrays with the x/y probabilities
    # and calculate the KL sum
    for f in fArr:
        Px.append(Gr.node[f]["x"])
        Py.append(Gr.node[f]["y"])
        #print("a" if x < y else "b")
        #KLx += ((Px[f] * math.log(Px[f]/Qx[f]))+(Qx[f] * math.log(Qx[f]/Px[f]))) if Px[f] > 0 else 0
        #KLy += ((Py[f] * math.log(Py[f]/Qy[f]))+(Qy[f] * math.log(Qy[f]/Py[f]))) if Py[f] > 0 else 0

    KLx = scipy.stats.entropy(Px, Qx)
    KLy = scipy.stats.entropy(Py, Qy)

    StoreGraphToFile(Gr, i, 2, "Reposition Files")
    
    # Assign the P sets to the Q sets for the next run
    Qx = Px
    Qy = Py
    Px = []
    Py = []
    
    print(str(i) + ": " + str(KLx) + ", " + str(KLy))
    i+=1

numIter = i-1

print("done in %0.3fs." % (time() - t0))


0: 0.290765119235, 0.238928124581
1: 0.0028378821003, 0.00235341428734
2: 0.000213198224923, 0.000178912570858
3: 2.87776315018e-05, 2.42051208815e-05
4: 5.90806078799e-06, 4.58296935101e-06
5: 1.68162903861e-06, 1.06799920528e-06
6: 5.96757365842e-07, 2.84712362614e-07
7: 2.40682643492e-07, 8.3806510889e-08
8: 1.04105181901e-07, 2.68687805488e-08
9: 4.68213118815e-08, 9.34249975797e-09
10: 2.15492334226e-08, 3.51067450924e-09
11: 1.0063539824e-08, 1.41507928292e-09
done in 7.077s.


In [993]:
# Extract values for Tree from Graph
Y = [];
Y_label = [];

for f in fArr:
    x = Gr.node[f]["x"]
    y = Gr.node[f]["y"]
    lab = Gr.node[f]["label"]
    Y.append([x,y])
    Y_label.append(lab)

tree = KDTree(Y)

In [994]:
t0 = time()

# Write test results to a file
filename = 'C:\\BBC_Data\\Pred9_News_' + str(int(_LK_Limit*10000000000)) + '_' + str(numIter) + '_' + str(_Test_Run) + '.txt'
myfile = open(filename, 'w')
wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)

for test_k in range(1,100):
    Actual = []
    Pred = []

    for f in fTest:
        sumx=0
        sumy=0
        sumw=0
        # What is the test set label?
        lab=GTest.node[f]["label"]
        pred_lab = dict()

        for t in GTest.neighbors(f):
            # is this test term in our term array?
            if t in tArr:
                # Get the weight of the edge from the test graph
                w=float(GTest.edge[f][t]["weight"])
                sumw+=w
                # Get the x,y of the term from the model graph
                sumx+=float(Gr.node[t]["x"])*w
                sumy+=float(Gr.node[t]["y"])*w

        if sumw > 0:
            # Get the centroid
            x = sumx/sumw
            y = sumy/sumw
            # Query the tree for nearest neighbor
            dist,ind = tree.query([[x,y]], k=test_k)
            for i in ind[0]:
                if Y_label[i] in pred_lab:
                    pred_lab[Y_label[i]] += 1
                else:
                    pred_lab[Y_label[i]] = 1

        else:
            pred_lab["UNK"] = 1

        #print(f + ": " + lab + ", pred=" + str(sorted(pred_lab.items(), key=operator.itemgetter(1), reverse=True)[0][0]))
        Actual.append(lab)
        Pred.append(str(sorted(pred_lab.items(), key=operator.itemgetter(1), reverse=True)[0][0]))

    precision = precision_score(Actual, Pred, average='weighted') 
    recall = recall_score(Actual, Pred, average='weighted')
    f1 = f1_score(Actual, Pred, average='weighted') 
    #print(str(test_k) + ',' + str(precision) + ',' + str(recall) + ',' + str(f1))
    
    myfile.write(str(test_k) + ',' + str(precision) + ',' + str(recall) + ',' + str(f1) +'\n')

myfile.close()

print("File written in %0.3fs." % (time() - t0))


File written in 106.546s.


In [ ]:
#################################
WAIT HERE
#################################

In [705]:
# Specify K to use for this test
test_k = 35

text = input()

#text = 'in natural language processing (NLP) a text graph is a graph representation of a text item '
#text += '(document, passage or sentence) it is typically created as a preprocessing step to support '
#text += 'NLP tasks such as text condensation term disambiguation (topic based) text summarization '
#text += '(summarize large text collections) and relation extraction (extract relations from unstructured text)'
d = ['dummy', text]

ftg = FileToGraph(d)
sumx=0
sumy=0
sumw=0
for res in ftg: 
    tm = res.split('\t')[1]
    wt = int(res.split('\t')[2])
    
    
    # Only include keywords
    if tm in tArr and wt>_degree_compare:
        sumw+=wt
        # Get the x,y of the term from the model graph
        x = Gr.node[tm]["x"]
        y = Gr.node[tm]["y"]

        print(tm + ': ' + str(wt) + ' (' + str(x) + ',' + str(y) + ')')

        sumx+=float(Gr.node[tm]["x"])*wt
        sumy+=float(Gr.node[tm]["y"])*wt

if sumw > 0:
    # Get the centroid
    x = sumx/sumw
    y = sumy/sumw
    print('Centroid: (' + str(x) + ',' + str(y) + ')')
    # Query the tree for nearest neighbor
    dist,ind = tree.query([[x,y]], k=test_k)
    for i in ind[0]:
        if Y_label[i] in pred_lab:
            pred_lab[Y_label[i]] += 1
        else:
            pred_lab[Y_label[i]] = 1

else:
    pred_lab["UNK"] = 1

print('Prediction: ' + str(pred_lab))
#str(sorted(pred_lab.items(), key=operator.itemgetter(1), reverse=True)[0][0]))



ad


IndexError: list index out of range

In [122]:
##### CLUSTERING


# Change text label to number
def strToNum(str):
    rVal = 0
    if str == "B":
        rVal = 0
    elif str == "E":
        rVal = 1
    elif str == "P":
        rVal = 2
    elif str == "S":
        rVal = 3
    elif str == "T":
        rVal = 4
        
    return rVal


# Load the data from the graph into a dataframe
df = pd.DataFrame(columns=['filename','x','y','Label'])
df.Label = df.Label.astype(int)

for f in fArr:
    s = strToNum(f[32:33])
    df = df.append({"filename": f, "x":Gr.node[f]["x"], "y":Gr.node[f]["y"], "Label": s}, ignore_index=True)

#df = df.append({"filename": "file1", "x":1, "y":2}, ignore_index=True)
#df = df.append({"filename": "file2", "x":2, "y":3}, ignore_index=True)

In [123]:
# Cluster the documents using kmeans
kmeans = KMeans(n_clusters=5,init='k-means++')
result = kmeans.fit_predict(df[["x","y"]])

In [124]:
df["Cluster"]=result

4

In [127]:
cm = confusion_matrix(df.Label, df.Cluster)

In [128]:
cm

array([[213,   0,   0,   2, 295],
       [ 79,   2,   0, 293,  12],
       [402,   0,   1,   4,  10],
       [ 47,   1, 457,   6,   0],
       [ 81,   0,   0,  12, 308]])

In [151]:

tc = pd.DataFrame(columns=['term','weight','Cluster'])
tc.Cluster = tc.Cluster.astype(int)

for f in fArr:
    dftemp=df[df['filename'] == f]
    c = dftemp.get_value(dftemp.index[0], col='Cluster')
    for t in Gr.neighbors(f):
        w=float(Gr.edge[f][t]["weight"])
        tc = tc.append({"term": t, "weight": w, "Cluster": c}, ignore_index=True)
        
        

In [556]:

numIter = 10